# Projet

### Imports

In [4]:
# Imports
import glob, os
import pandas
import urllib
from SPARQLWrapper import SPARQLWrapper
import PIL.Image

## 1. Collecte de données

### Récupération du résultat d'une query Wikidata

In [5]:
size = 5
dataFileName = "data"
imagesPath = "./images/"

# Query de Wikidata
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
query = """
SELECT ?item ?itemLabel ?pic WHERE {
  ?item wdt:P31 wd:Q146;
    wdt:P18 ?pic.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT """
# On précise le nombre d'entrées que l'on veut
query = query + str(size)

# On récupère le résultat de la query
sparql.setQuery(query)
sparql.setReturnFormat("json")
results = sparql.query().convert()

# On normalise les résultat pour pandas
array = []
for data in results["results"]["bindings"]:
    name = data["itemLabel"]["value"]
    format = data["pic"]["value"].split(".")[-1]
    path = f"{imagesPath}{name}.{format}"

    array.append([name,  
                  data["pic"]["value"], 
                  data["item"]["value"],
                  format,
                  path,
                  None])
                  

dataframe = pandas.DataFrame(array, columns=["label", "link", "wikiDataRef", "format", "path", "size"])

# On enregistre les données au format JSON
with open(dataFileName+'.json', "w", encoding="utf-8") as f:
    f.write(dataframe.to_json(orient='index', indent=4))

# Nettoyage du dossier /images
if not os.path.isdir('./images'):
  os.mkdir('./images')

# Nettoyage du dossier /images
else:
  files = glob.glob('./images/*')
  for f in files:
      os.remove(f)



dataframe


,label,link,wikiDataRef,format,path,size
0,senior cats,http://commons.wikimedia.org/wiki/Special:File...,http://www.wikidata.org/entity/Q123185365,jpg,./images/senior cats.jpg,None
1,Myka,http://commons.wikimedia.org/wiki/Special:File...,http://www.wikidata.org/entity/Q28665865,jpg,./images/Myka.jpg,None
2,Gli,http://commons.wikimedia.org/wiki/Special:File...,http://www.wikidata.org/entity/Q28792126,jpg,./images/Gli.jpg,None
3,Toffee,http://commons.wikimedia.org/wiki/Special:File...,http://www.wikidata.org/entity/Q49581026,jpg,./images/Toffee.jpg,None
4,Crimean Tom,http://commons.wikimedia.org/wiki/Special:File...,http://www.wikidata.org/entity/Q51596094,jpg,./images/Crimean Tom.jpg,None


### Téléchargement des Images

In [6]:
for i in range(len(dataframe.index)):
    # On télécharge l'image
    link = dataframe._get_value(i,"link")
    path = dataframe._get_value(i,"path")
    response = urllib.request.urlretrieve(link, path)

### Récupération des Données EXIFS

In [7]:
wantedExifs = [274, 315, 306]
columnNames = ["orientation", "artist", "date"]

for i,tag in enumerate(wantedExifs):
    dataframe[columnNames[i]] = None

# On parcours les entrées
for i in range(len(dataframe.index)):
    path = dataframe._get_value(i,"path")
    img = PIL.Image.open(path)
    exif_data = img._getexif()
    img.close()

    dataframe.at[i, "size"] = img.size
    if exif_data:
      for tag in exif_data:
          if tag in wantedExifs:
              dataframe.at[i, columnNames[wantedExifs.index(tag)]] = exif_data[tag]

# On enregistre les données au format JSON
with open(dataFileName+'.json', "w", encoding="utf-8") as f:
    f.write(dataframe.to_json(orient='index', indent=4))
        
dataframe 

NameError: name 'saveDataFrame' is not defined